# MODELING

In [1]:
from tqdm import tqdm
from collections import defaultdict
import gensim
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models import LdaModel
import pyLDAvis.gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymystem3 import Mystem
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import seaborn as sns
%matplotlib inline

/Users/wicks/anaconda/lib/python3.6/site-packages/nltk/decorators.py:59: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  regargs, varargs, varkwargs, defaults = inspect.getargspec(func)


# Select the neighborhood and load its data

In [2]:
name = 'Izmajlovo'
district = gpd.read_file('../data/Izmajlovo_district.geojson'.format(name,name))
neigh_posts = pd.read_csv('../data/vk.csv'.format(name))

In [3]:
neigh_posts

,Unnamed: 0,lat,lon,userId,date,text,place_id,hour,minutes,day,weekday,geometry
0,142520,55.772002,37.750965,15315356,1464748293,С Динарой #подружки #девчонки 😊👯,8896,18000,31,1,2,POINT (37.750964984663 55.772002353841)
1,142521,55.772002,37.750965,198980737,1464619527,Очень круто погуляли✌️,8896,61200,45,30,0,POINT (37.750964984663 55.772002353841)
2,142522,55.772002,37.750965,27391343,1464560462,#izmailovo_may2016,8896,3600,21,29,6,POINT (37.750964984663 55.772002353841)
3,142523,55.772002,37.750965,19597517,1464534383,Погода класс!!!,8896,64800,6,29,6,POINT (37.750964984663 55.772002353841)
4,142524,55.772002,37.750965,175687971,1464452173,Катаемся в #измайловопарк,8896,68400,16,28,5,POINT (37.750964984663 55.772002353841)
5,142525,55.772002,37.750965,5890158,1463851581,От куда такая нежность..#этовеснадетка,8896,72000,26,21,5,POINT (37.750964984663 55.772002353841)
6,142526,55.772002,37.750965,5890158,1463849082,"Какой прекрасный день, какой прекрасный парк.....",8896,68400,44,21,5,POINT (37.750964984663 55.772002353841)
7,142527,55.772002,37.750965,9405635,1463611735,Измайловский парк🐾,8896,3600,48,18,2,POINT (37.750964984663 55.772002353841)
8,142528,55.777012,37.789018,1116591,1464748285,Последний закат весны - 2016.<br>Поздравляю вс...,8897,18000,31,1,2,POINT (37.789018159453 55.77701215524399)
9,142529,55.777012,37.789018,236002945,1464744989,С Днем Защиты Детей!!!Берегите в себе ребенка)...,8897,14400,36,1,2,POINT (37.789018159453 55.77701215524399)


In [4]:
len(neigh_posts)

36104

# Group posts by user

In [5]:
vk_users = pd.DataFrame({'post' : neigh_posts.groupby('userId').apply( lambda x: ' '.join(x['text']))}).reset_index()

In [6]:
vk_users

,userId,post
0,592,Мужской тренинг Павла Ракова. Круто! Рекоменд...
1,967,#люблюподарки #холоднобля #СЕКСШМЕКС @ Измайло...
2,1666,Главное в праздничном ужине... Вам понадобится...
3,2156,Трава поседела...#Осень#иней @ Измайловский парк
4,3042,День России 🇷🇺🇷🇺🇷🇺#япатриотка @ Круглый Пруд ...
5,4505,@fitness_marcus_aurelius #МаркАврелий #Вешняк...
6,4742,Готовимся к Зомби-забегу! <br><br>Измайловский...
7,5286,"Интересный проект, в рамках которого дети позн..."
8,5294,Последние солнечные деньки осени! Наслаждайтес...
9,6703,💞 Вы лучшие! @sadatgulu @salikks Большая радос...


In [32]:
# How many users are there?

In [7]:
len(vk_users)

17369

In [8]:
extra_words = ['http','br','id','com','www', 'instagram', 'vsco', 'https', 'instasize','repost',
              'whatsapp', 'вотсап', 'repostapp']
def process_docs(docs):
    """
    Function to process texts. Following are the steps we take:
    
    1. Text tokenization.
    2. Removing numbers 
    3. Stopword and short words Removal.
    4. Lemmatization and filter words by their length.
    
    Args:
    ----------
    texts: Tokenized texts.
    
    Returns:
    -------
    texts: Pre-processed tokenized texts.
    """
    m = Mystem()
    # Split the documents into tokens.
    tokenizer = RegexpTokenizer(r'\w+')
    stops = stopwords.words('russian') + stopwords.words('english') + extra_words
    
    for idx in tqdm(range(len(docs))):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not any(c.isdigit() for c in token) and ('id' not in token or 'club' not in token or 'ru' not in token)] for doc in tqdm(docs)]
    #Lemmatize words
    docs = [[m.lemmatize(token)[0] for token in doc ] for doc in tqdm(docs)]
    #Remove stopwords
    docs = [[token for token in doc if token not in stops] for doc in tqdm(docs)]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in tqdm(docs)]
    return docs
    
def get_corpus(docs):
    
    """Add bigrams to docs and create corpus and dictionary for training
    
    Args:
        docs: list of tokenized and cleaned texts;
    Returns:
        corpus: list of lists of tuples, where first element of tuple is a word id
        and the second is the count of that word in the whole corpus
        dictionary: gensim.corpora.dictionary.Dictionary 
  
    """
    
    frequency = defaultdict(int)
    for text in tqdm(docs):
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 3] for text in tqdm(docs)]

    #Take the bigram, if token is a bigram, add to document.
    bigram = Phrases(texts, min_count = 20)
    for idx in tqdm(range(len(texts))):
        for token in bigram[texts[idx]]:
            if '_' in token:
                texts[idx].append(token)
    
    # Create a dictionary representation of the documents.
    dictionary = Dictionary(texts)
    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    #dictionary.filter_extremes(no_below=20, no_above=0.5)
    corpus = [dictionary.doc2bow(doc) for doc in tqdm(texts)]
    
    print('Number of unique tokens: {}'.format(len(dictionary)))
    print('Number of documents: {}'.format(len(corpus)))
    
    return corpus, dictionary

# Process texts to be ready for modeling

In [9]:
texts = vk_users['post'].copy()

In [10]:
docs = process_docs(texts.values)

100%|██████████| 17369/17369 [00:00<00:00, 282592.24it/s]


In [11]:
corpus, dictionary = get_corpus(docs)

  0%|          | 0/17369 [00:00<?, ?it/s]/Users/wicks/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
100%|██████████| 17369/17369 [00:00<00:00, 42785.65it/s]

Number of unique tokens: 8563
Number of documents: 17369


In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


#The training model - we use online LDA model which allows to update the model 
#and the following parameters should be defined
num_topics = 10 # number of topics
chunksize = 1000 
passes = 10
iterations = 400
eval_every = 10  #evaluate model perplexity.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha=0.001, update_every = 1, \
                       num_topics=num_topics,\
                       eval_every=eval_every, passes = passes)

2017-07-19 10:42:22,361 : INFO : using symmetric eta at 0.00011678150181011328
2017-07-19 10:42:22,364 : INFO : using serial LDA version on this node
2017-07-19 10:42:22,840 : INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 17369 documents, updating model once every 1000 documents, evaluating perplexity every 10000 documents, iterating 50x with a convergence threshold of 0.001000
2017-07-19 10:42:22,841 : INFO : PROGRESS: pass 0, at document #1000/17369
2017-07-19 10:42:23,778 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-19 10:42:23,823 : INFO : topic #5 (0.001): 0.019*"hotel" + 0.019*"измайлово" + 0.018*"vega" + 0.016*"convention" + 0.016*"best" + 0.016*"best_western" + 0.015*"plus" + 0.015*"western" + 0.015*"center" + 0.015*"измайловский"
2017-07-19 10:42:23,824 : INFO : topic #9 (0.001): 0.014*"спасибо" + 0.011*"hotel" + 0.011*"москва" + 0.011*"день" + 0.010*"western" + 0.010*"измайлово" + 0.010*"vega" + 0.

2017-07-19 10:42:27,181 : INFO : topic #7 (0.001): 0.075*"измайлово" + 0.044*"дельта" + 0.043*"гамма" + 0.043*"измайлово_гамма" + 0.014*"moscow" + 0.012*"концертный" + 0.011*"концертный_измайлово" + 0.011*"москва" + 0.006*"square" + 0.006*"square_moscow"
2017-07-19 10:42:27,182 : INFO : topic #9 (0.001): 0.025*"ргуфксить" + 0.018*"спасибо" + 0.018*"день" + 0.012*"человек" + 0.010*"новый" + 0.009*"свой" + 0.007*"каждый" + 0.007*"друг" + 0.007*"рождение" + 0.007*"день_рождение"
2017-07-19 10:42:27,183 : INFO : topic #1 (0.001): 0.028*"измайлово" + 0.020*"альфа" + 0.019*"гостиница" + 0.018*"гостиница_измайлово" + 0.017*"izmaylovo" + 0.016*"измайловский" + 0.014*"district" + 0.013*"motostudy" + 0.012*"izmaylovo_district" + 0.011*"платье"
2017-07-19 10:42:27,184 : INFO : topic #5 (0.001): 0.048*"hotel" + 0.043*"vega" + 0.038*"best" + 0.035*"best_western" + 0.035*"western" + 0.035*"center" + 0.035*"convention" + 0.034*"plus" + 0.033*"hotel_convention" + 0.033*"plus_vega"
2017-07-19 10:42:27,

2017-07-19 10:42:30,708 : INFO : topic #5 (0.001): 0.061*"hotel" + 0.056*"vega" + 0.048*"best" + 0.046*"best_western" + 0.046*"western" + 0.046*"center" + 0.045*"convention" + 0.045*"plus" + 0.045*"plus_vega" + 0.045*"hotel_convention"
2017-07-19 10:42:30,709 : INFO : topic diff=0.428449, rho=0.301511
2017-07-19 10:42:30,710 : INFO : PROGRESS: pass 0, at document #12000/17369
2017-07-19 10:42:31,133 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-19 10:42:31,192 : INFO : topic #4 (0.001): 0.019*"утро" + 0.019*"hotel" + 0.015*"izmailovo" + 0.014*"москва" + 0.013*"добрый" + 0.010*"день" + 0.009*"beta" + 0.009*"добрый_утро" + 0.009*"genius" + 0.009*"genius_beauty"
2017-07-19 10:42:31,193 : INFO : topic #0 (0.001): 0.060*"москва" + 0.034*"park" + 0.029*"izmaylovsky" + 0.029*"izmaylovsky_park" + 0.024*"красный" + 0.023*"площадь" + 0.021*"фото" + 0.019*"москва_красный" + 0.019*"moscow" + 0.018*"опубликовывать"
2017-07-19 10:42:31,194 : INFO : topic #6 (0.

2017-07-19 10:42:33,467 : INFO : topic #6 (0.001): 0.031*"парк" + 0.027*"сокольники" + 0.023*"парк_сокольники" + 0.019*"пруд" + 0.011*"мюзикл" + 0.010*"круглый" + 0.010*"круглый_пруд" + 0.009*"ргуфк" + 0.008*"гцолифк" + 0.008*"виноградный"
2017-07-19 10:42:33,468 : INFO : topic #0 (0.001): 0.076*"москва" + 0.037*"park" + 0.032*"izmaylovsky" + 0.032*"izmaylovsky_park" + 0.025*"красный" + 0.025*"moscow" + 0.024*"площадь" + 0.023*"фото" + 0.019*"москва_красный" + 0.019*"опубликовывать"
2017-07-19 10:42:33,469 : INFO : topic #2 (0.001): 0.172*"измайловский" + 0.081*"парк" + 0.072*"измайловский_парк" + 0.033*"измайлово" + 0.032*"остров" + 0.030*"измайловский_остров" + 0.028*"усадьба" + 0.024*"усадьба_измайлово" + 0.018*"бульвар" + 0.017*"измайловский_бульвар"
2017-07-19 10:42:33,470 : INFO : topic diff=0.288805, rho=0.242536
2017-07-19 10:42:33,853 : INFO : -9.089 per-word bound, 544.4 perplexity estimate based on a held-out corpus of 369 documents with 8646 words
2017-07-19 10:42:33,854 : 

2017-07-19 10:42:35,806 : INFO : topic diff=0.196142, rho=0.227220
2017-07-19 10:42:35,807 : INFO : PROGRESS: pass 1, at document #5000/17369
2017-07-19 10:42:36,192 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-19 10:42:36,228 : INFO : topic #1 (0.001): 0.034*"измайлово" + 0.033*"izmaylovo" + 0.030*"district" + 0.030*"альфа" + 0.029*"гостиница" + 0.029*"izmaylovo_district" + 0.026*"гостиница_измайлово" + 0.012*"москва" + 0.011*"фитнес" + 0.009*"измайловский"
2017-07-19 10:42:36,229 : INFO : topic #5 (0.001): 0.051*"hotel" + 0.049*"vega" + 0.041*"best" + 0.039*"center" + 0.039*"western" + 0.039*"best_western" + 0.039*"convention" + 0.037*"plus" + 0.037*"plus_vega" + 0.037*"hotel_convention"
2017-07-19 10:42:36,230 : INFO : topic #6 (0.001): 0.030*"парк" + 0.028*"сокольники" + 0.022*"парк_сокольники" + 0.019*"пруд" + 0.011*"круглый" + 0.010*"круглый_пруд" + 0.008*"ребята" + 0.008*"мюзикл" + 0.007*"casa" + 0.007*"бассейн"
2017-07-19 10:42:36,231 : I

...

2017-07-19 10:43:40,856 : INFO : topic #4 (0.001): 0.025*"утро" + 0.023*"москва" + 0.020*"hotel" + 0.017*"добрый" + 0.017*"izmailovo" + 0.013*"день" + 0.011*"beta" + 0.011*"работа" + 0.011*"добрый_утро" + 0.010*"хороший"
2017-07-19 10:43:40,857 : INFO : topic #9 (0.001): 0.019*"день" + 0.014*"спасибо" + 0.012*"ргуфксить" + 0.012*"свой" + 0.011*"самый" + 0.010*"новый" + 0.010*"человек" + 0.010*"вопрос" + 0.010*"любить" + 0.010*"заказ"
2017-07-19 10:43:40,858 : INFO : topic #2 (0.001): 0.181*"измайловский" + 0.084*"парк" + 0.076*"измайловский_парк" + 0.033*"измайлово" + 0.026*"остров" + 0.026*"усадьба" + 0.024*"измайловский_остров" + 0.021*"усадьба_измайлово" + 0.018*"бульвар" + 0.018*"измайловский_бульвар"
2017-07-19 10:43:40,859 : INFO : topic #6 (0.001): 0.027*"сокольники" + 0.027*"парк" + 0.025*"ргуфксить" + 0.023*"парк_сокольники" + 0.017*"пруд" + 0.011*"ргуфк" + 0.010*"мюзикл" + 0.008*"круглый" + 0.008*"спорт" + 0.008*"гцолифк"
2017-07-19 10:43:40,860 : INFO : topic diff=0.119207, 

2017-07-19 10:43:42,863 : INFO : topic diff=0.063752, rho=0.191148
2017-07-19 10:43:42,864 : INFO : PROGRESS: pass 9, at document #6000/17369
2017-07-19 10:43:43,239 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-19 10:43:43,257 : INFO : topic #6 (0.001): 0.029*"парк" + 0.027*"сокольники" + 0.022*"парк_сокольники" + 0.019*"пруд" + 0.019*"ргуфксить" + 0.010*"круглый" + 0.009*"круглый_пруд" + 0.009*"ребенок" + 0.009*"мюзикл" + 0.007*"бассейн"
2017-07-19 10:43:43,258 : INFO : topic #4 (0.001): 0.023*"утро" + 0.019*"москва" + 0.015*"hotel" + 0.014*"добрый" + 0.013*"izmailovo" + 0.012*"день" + 0.011*"платье" + 0.011*"сегодня" + 0.009*"хороший" + 0.009*"добрый_утро"
2017-07-19 10:43:43,259 : INFO : topic #9 (0.001): 0.020*"день" + 0.016*"спасибо" + 0.012*"свой" + 0.011*"человек" + 0.010*"ргуфксить" + 0.010*"самый" + 0.010*"новый" + 0.009*"любить" + 0.009*"сегодня" + 0.009*"очень"
2017-07-19 10:43:43,260 : INFO : topic #2 (0.001): 0.183*"измайловский" + 0

2017-07-19 10:43:46,133 : INFO : topic #4 (0.001): 0.024*"утро" + 0.021*"москва" + 0.017*"hotel" + 0.014*"izmailovo" + 0.014*"добрый" + 0.012*"день" + 0.011*"сегодня" + 0.010*"хороший" + 0.010*"работа" + 0.009*"beta"
2017-07-19 10:43:46,134 : INFO : topic #9 (0.001): 0.020*"день" + 0.018*"спасибо" + 0.014*"ргуфксить" + 0.013*"свой" + 0.012*"человек" + 0.011*"самый" + 0.011*"любить" + 0.010*"сегодня" + 0.010*"который" + 0.009*"новый"
2017-07-19 10:43:46,135 : INFO : topic #1 (0.001): 0.047*"измайлово" + 0.036*"гостиница" + 0.034*"альфа" + 0.031*"гостиница_измайлово" + 0.031*"izmaylovo" + 0.026*"district" + 0.024*"izmaylovo_district" + 0.019*"зверинец" + 0.019*"измайловский_зверинец" + 0.013*"автоэлектрик"
2017-07-19 10:43:46,136 : INFO : topic diff=0.077976, rho=0.191148
2017-07-19 10:43:46,136 : INFO : PROGRESS: pass 9, at document #12000/17369
2017-07-19 10:43:46,490 : INFO : merging changes from 1000 documents into a model of 17369 documents
2017-07-19 10:43:46,509 : INFO : topic #0 

2017-07-19 10:43:48,481 : INFO : topic #8 (0.001): 0.145*"измайлово" + 0.144*"кремль" + 0.107*"кремль_измайлово" + 0.022*"make" + 0.022*"atelier" + 0.020*"school" + 0.020*"make_atelier" + 0.017*"москва" + 0.013*"kremlin" + 0.012*"izmailovo"
2017-07-19 10:43:48,482 : INFO : topic #9 (0.001): 0.019*"день" + 0.016*"спасибо" + 0.014*"ргуфксить" + 0.012*"свой" + 0.011*"человек" + 0.011*"любить" + 0.011*"самый" + 0.010*"писать" + 0.010*"цена" + 0.010*"вопрос"
2017-07-19 10:43:48,483 : INFO : topic #0 (0.001): 0.082*"москва" + 0.039*"park" + 0.034*"izmaylovsky_park" + 0.034*"izmaylovsky" + 0.032*"moscow" + 0.027*"красный" + 0.027*"фото" + 0.025*"площадь" + 0.021*"опубликовывать" + 0.020*"опубликовывать_фото"
2017-07-19 10:43:48,484 : INFO : topic #1 (0.001): 0.052*"измайлово" + 0.045*"альфа" + 0.043*"гостиница" + 0.042*"izmaylovo" + 0.037*"district" + 0.036*"izmaylovo_district" + 0.036*"гостиница_измайлово" + 0.014*"москва" + 0.012*"отель" + 0.010*"фитнес"
2017-07-19 10:43:48,485 : INFO : top

CPU times: user 1min 26s, sys: 795 ms, total: 1min 26s
Wall time: 1min 26s


In [13]:
import nltk
# nltk.download()

In [14]:
data = pyLDAvis.gensim.prepare(model, corpus, dictionary); # visualize lda topics
pyLDAvis.display(data)

# Assign topics to users

In [15]:
docTopicProbMat = model[corpus]
lda_users = vk_users.copy()
lda_users['topics'] = docTopicProbMat
vk_users['topic'] = lda_users['topics'].apply(lambda x :x[0][0])

# Assign topics to their posts

In [16]:
neigh_posts['topic'] = neigh_posts['userId'].apply(lambda userId: vk_users.loc[vk_users['userId']==userId,'topic'].item())

# Plot the histogram of topic distribution per posts and per users

In [17]:
neigh_posts

,Unnamed: 0,lat,lon,userId,date,text,place_id,hour,minutes,day,weekday,geometry,topic
0,142520,55.772002,37.750965,15315356,1464748293,С Динарой #подружки #девчонки 😊👯,8896,18000,31,1,2,POINT (37.750964984663 55.772002353841),2
1,142521,55.772002,37.750965,198980737,1464619527,Очень круто погуляли✌️,8896,61200,45,30,0,POINT (37.750964984663 55.772002353841),4
2,142522,55.772002,37.750965,27391343,1464560462,#izmailovo_may2016,8896,3600,21,29,6,POINT (37.750964984663 55.772002353841),0
3,142523,55.772002,37.750965,19597517,1464534383,Погода класс!!!,8896,64800,6,29,6,POINT (37.750964984663 55.772002353841),6
4,142524,55.772002,37.750965,175687971,1464452173,Катаемся в #измайловопарк,8896,68400,16,28,5,POINT (37.750964984663 55.772002353841),0
5,142525,55.772002,37.750965,5890158,1463851581,От куда такая нежность..#этовеснадетка,8896,72000,26,21,5,POINT (37.750964984663 55.772002353841),1
6,142526,55.772002,37.750965,5890158,1463849082,"Какой прекрасный день, какой прекрасный парк.....",8896,68400,44,21,5,POINT (37.750964984663 55.772002353841),1
7,142527,55.772002,37.750965,9405635,1463611735,Измайловский парк🐾,8896,3600,48,18,2,POINT (37.750964984663 55.772002353841),2
8,142528,55.777012,37.789018,1116591,1464748285,Последний закат весны - 2016.<br>Поздравляю вс...,8897,18000,31,1,2,POINT (37.789018159453 55.77701215524399),0
9,142529,55.777012,37.789018,236002945,1464744989,С Днем Защиты Детей!!!Берегите в себе ребенка)...,8897,14400,36,1,2,POINT (37.789018159453 55.77701215524399),0


# Mapping

In [18]:
center_lat = list(district.centroid[0].coords)[0][1]
center_lon = list(district.centroid[0].coords)[0][0]

In [28]:
map_places = folium.Map([center_lat, center_lon], tiles='Stamen Toner', zoom_start=14,control_scale=True)

#Define style for geojson objects
style_function = lambda feature: dict(fillColor='#AECCAE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.3)

# Adding Houses
houses = gpd.read_file('../data/Izmajlovo_chruchevki.geojson'.format(name, name))
points = folium.features.GeoJson(houses,name='Khurshevki houses')
map_places.add_child(points,name='Khurshevki houses')

# Adding Houses with markers
houses = gpd.read_file('../data/Izmajlovo_chruchevki.geojson'.format(name, name))
points = folium.features.GeoJson(houses,name='Khurshevki houses')
map_places.add_child(points,name='Khurshevki houses')

#Adding district
polygon = folium.features.GeoJson(district, style_function=style_function,name='district boundary')
map_places.add_child(polygon,name='district boundary')

colormap_dict = {0.0: 'pink', 0.3: 'blue', 0.5: 'green',  1.0: 'red'}

#Adding topics heatmaps
for topic_id in [1, 2, 3, 4]:
    topic_coords = list(zip(neigh_posts[neigh_posts.topic == topic_id].lat, neigh_posts[neigh_posts.topic == topic_id].lon))

    HeatMap(topic_coords,
            name='Topic: {}'.format(topic_id),
            radius=10, 
            min_opacity=0.8,
            gradient={0.0: 'blue', 0.3: 'cyan', 0.5: 'yellow',  1.0: 'red'}).add_to(map_places)

    colormap = folium.LinearColormap(colors = colormap_dict.values())
    colormap.caption = 'Topic: {}'.format(topic_id)

#Switch between layers
folium.LayerControl().add_to(map_places)
map_places

In [ ]:
import nltk
nltk.download()